In [ ]:
!nvidia-smi

Fri Jul 23 17:39:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install -qq tokenizers===0.10.3

     |████████████████████████████████| 3.3 MB 7.6 MB/s 


In [ ]:
!pip install -qq transformers

     |████████████████████████████████| 2.6 MB 7.3 MB/s 
     |████████████████████████████████| 636 kB 43.2 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 


In [ ]:
!pip install -qq simpletransformers

     |████████████████████████████████| 221 kB 8.6 MB/s 
     |████████████████████████████████| 1.8 MB 60.7 MB/s 
     |████████████████████████████████| 124 kB 76.3 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 7.9 MB 23.2 MB/s 
     |████████████████████████████████| 1.2 MB 63.4 MB/s 
     |████████████████████████████████| 76 kB 5.8 MB/s 
     |████████████████████████████████| 542 kB 69.0 MB/s 
     |████████████████████████████████| 170 kB 68.0 MB/s 
     |████████████████████████████████| 133 kB 79.1 MB/s 
     |████████████████████████████████| 138 kB 46.1 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 243 kB 61.4 MB/s 
     |████████████████████████████████| 118 kB 76.8 MB/s 
     |████████████████████████████████| 111 kB 76.3 MB/s 
     |███████████████████

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
pretrained = "monsoon-nlp/bert-base-thai"

In [ ]:
model_args = {
    'manual_seed' : 0,
    'reprocess_input_data': False,
    
    'train_batch_size': 16,
    'eval_batch_size' : 8,
    
    'use_early_stopping': True,
    'early_stopping_delta': 0.01,
    'early_stopping_metric': 'eval_loss',
    'early_stopping_metric_minimize': True,
    'early_stopping_patience': 5,
    
    'evaluate_during_training' : True,
    'evaluate_during_training_verbose': True,
    
    'fp16': True,
    
    'num_train_epochs': 10,
    
    'overwrite_output_dir': True,

    'save_model_every_epoch': False,
    'save_steps':-1,
    
    'use_cached_eval_features' : True, 
    
    'max_seq_length': 256,
    'no_cache': False,

    'custom_parameter_groups':[{
        'params': ['classifier.weight', 'bert.encoder.layer.10.output.dense.weight'],
        'lr': 5e-5,
    }]
}

In [ ]:
bert = QuestionAnsweringModel(
    'bert', 
    pretrained,
    use_cuda=torch.cuda.is_available(), 
    args=model_args
)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from th

In [ ]:
import json
with open('scg/train.json', 'r') as file:
  scgtrain = json.load(file, encoding='utf-8')
with open('scg/eval.json', 'r') as file:
  scgeval = json.load(file, encoding='utf-8')
with open('wiki/train.json','r') as file:
  wikitrain = json.load(file, encoding='utf-8')
with open('wiki/eval.json','r') as file:
  wikieval = json.load(file, encoding='utf-8')
train = scgtrain + wikitrain
eval = scgeval + wikieval

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |    2740 MB |    1999 GB |    1999 GB |\n|       from large pool |       0 B  |    2719 MB |    1990 GB |    1990 GB |\n|       from small pool |       0 B  |      21 MB |       8 GB |       8 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |    2740 MB |    1999 GB |    1999 GB |\n|       from large pool |       0 B  |    2719 MB |

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7410 sha256=bbd2ba5ea6b9f9c1a8379d02a0c55254768787c8b9ed25a6308772ea9901beca
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Gen RAM Free: 12.0 GB  |     Proc size: 1.5 GB
GPU RAM Free: 15106MB | Used: 3MB | Util   0% | Total     15109MB


In [ ]:
train = train[0:1000]

In [ ]:
eval = eval[0:200]

In [ ]:
step, train_result = bert.train_model(train, eval_data=eval, show_running_loss=True)


convert squad examples to features: 100%|██████████| 2089/2089 [00:51<00:00, 40.31it/s]  

add example index and unique id: 100%|██████████| 2089/2089 [00:00<00:00, 484457.65it/s]



convert squad examples to features: 100%|██████████| 265/265 [00:03<00:00, 68.95it/s]

add example index and unique id: 100%|██████████| 265/265 [00:00<00:00, 242154.81it/s]


In [ ]:
eval_result, eval_texts = bert.eval_model(eval)


convert squad examples to features: 100%|██████████| 3042/3042 [01:07<00:00, 45.13it/s]   

add example index and unique id: 100%|██████████| 3042/3042 [00:00<00:00, 510342.50it/s]


In [ ]:
eval_result

{'correct': 0, 'eval_loss': -5.792228816105769, 'incorrect': 4, 'similar': 3}

In [ ]:
eval_texts

{'correct_text': {},
 'incorrect_text': {1: {'predicted': 'และห่างจากตัวเมือง',
   'question': 'ที่หาดกมลามีอนุสรณ์สถานสึนามิที่สร้างเพื่อรำลึกถึงเหตุการณ์ที่เกิดขึ้นเมื่อปี พ.ศ. ใด',
   'truth': '2547'},
  4: {'predicted': 'แสตมป์',
   'question': 'แสตมป์ดวงแรกที่มีข้อความแสดงเหตุการณ์ที่ระลึกคือแสตมป์ที่ออกมาเพื่อฉลองครบรอบร้อยปีของการก่อตั้งเมืองใด',
   'truth': 'นิวเซาท์เวลส์'},
  5: {'predicted': 'ฉันมาทำอะไรที่นี่ Phornthz',
   'question': 'ตัวอย่างระบบการดูแลพนักงานที่ Digital Office มีอะไรบ้าง',
   'truth': 'Health center ใจกลางบริษัทที่จะเดินไปหาหมอเมื่อไหร่ก็ได้'},
  6: {'predicted': 'หวานชะเอม แห่ง',
   'question': 'Moonshot คืออะไร',
   'truth': 'สร้างธุรกิจใหม่ด้วยนวัตกรรมใหม่ๆ'}},
 'similar_text': {2: {'predicted': 'อุทยานแห่งชาติแม่วงก์ อุทยานแห่งชาติแม่วงก์ มีพื้นที่ครอบคลุมท้องที่อำเภอปางศิลาทอง จังหวัดกำแพงเพชร และอำเภอแม่วงก์ และอำเภอแม่เปิน จังหวัดนครสวรรค์ พื้นที่ส่วนใหญ่เป็นแหล่งกำเนิดต้นน้ำลำธาร ตามเทือกเขาสูงชันก่อกำเนิดเป็นน้ำตกที่สวยงาม 4-5 แห่ง',
   'question

In [ ]:
with open('test.json', 'r') as file:
  test = json.load(file, encoding='utf-8')

In [ ]:
answers, probabilities = bert.predict([test[0]])


convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 58.04it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11096.04it/s]


In [ ]:
output = dict()
for i in test:
  answers,probabilities = bert.predict([i])
  for j in range(len(answers)):
    print(i['qas'][j]['question_id'])
    question_id = i['qas'][j]['question_id']
    answer = answers[j]['answer'][0]
    output[question_id] = [question_id,answer]


convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 70.59it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10433.59it/s]



5507



convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 73.11it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 7410.43it/s]



5523



convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 39.10it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10591.68it/s]


...


5160



convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 18.16it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 7626.01it/s]



5161



convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 15.37it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6909.89it/s]



5162


In [ ]:
output

{'1000': ['1000', ''],
 '1569': ['1569',
  'เป็นสัตว์ในตำนานของคติชนญี่ปุ่น มีหัวเป็นเสือและร่างเป็นปลาคาร์ฟที่ปกคลุมไปด้วยหนามแหลมมีพิษ เมื่ออยู่บนบกชะจิโฮะโกะสามารถแปลงตนเป็นเสือได้อย่างสมบูรณ์ เชื่อกันว่าชะจิโฮะโกะมีอำนาจที่สามารถดลบันดาลให้ฝนตกได้ ด้วยความเชื่อนี้เอง'],
 '1570': ['1570', 'พุทธศักราชที่ตรงกับปีมะเส็ง'],
 '1571': ['1571', 'ของรอบปีนักษัตร'],
 '1572': ['1572', 'พรรคคอมมิวนิสต์อินโดจีน'],
 '1573': ['1573',
  'วิทยาลัยชุมชนตาก วิทยาลัยชุมชนตาก \xa0เป็นสถาบันอุดมศึกษาของรัฐในกลุ่มสถาบันวิทยาลัยชุมชน ในสังกัดสำนักงานคณะกรรมการการอุดมศึกษา\xa0กระทรวงศึกษาธิการ ซึ่งเปิดสอนในหลักสูตรอนุปริญญา ประกาศนียบัตรวิชาชีพชั้นสูงและประกาศนียบัตรวิชาชีพ โดยการเสริมสร้างโอกาสทางการศึกษาระดับอุดมศึกษา เพื่อเพิ่มคุณค่าชีวิตและศักยภาพของบุคคลและชุมชนประวัติ ประวัติ. วิทยาลัยชุมชนตาก เป็นวิทยาลัยชุมชนในสังกัดสำนักงานคณะกรรมการการอุดมศึกษา กระทรวงศึกษาธิการ จัดตั้งขึ้นเมื่อวันที่ 17 เมษายน พ.ศ. 2545 เป็น 1 ใน 10 วิทยาลัยแรกตั้งของกระทรวงศึกษาธิการ โดยการคัดเลือกสถานศึกษาศูนย์การศึกษานอกโรงเร

In [ ]:
df = []
for i in range(1,5609):
  if str(i) in output:
    df.append([i,output[str(i)][1]])
  else:
    df.append([i,''])

[[1, ''],
 [2, ''],
 [3, ''],
 [4, ''],
 [5, ''],
 [6, ''],
 [7, ''],
 [8, ''],
 [9, ''],
 [10, ''],
 [11, ''],
 [12, ''],
 [13, ''],
 [14, ''],
 [15, ''],
 [16, ''],
 [17, ''],
 [18, ''],
 [19, ''],
 [20, ''],
 [21, ''],
 [22, ''],
 [23, ''],
 [24, ''],
 [25, ''],
 [26, ''],
 [27, ''],
 [28, ''],
 [29, ''],
 [30, ''],
 [31, ''],
 [32, ''],
 [33, ''],
 [34, ''],
 [35, ''],
 [36, ''],
 [37, ''],
 [38, ''],
 [39, ''],
 [40, ''],
 [41, ''],
 [42, ''],
 [43, ''],
 [44, ''],
 [45, ''],
 [46, ''],
 [47, ''],
 [48, ''],
 [49, ''],
 [50, ''],
 [51, ''],
 [52, ''],
 [53, ''],
 [54, ''],
 [55, ''],
 [56, ''],
 [57, ''],
 [58, ''],
 [59, ''],
 [60, ''],
 [61, ''],
 [62, ''],
 [63, ''],
 [64, ''],
 [65, ''],
 [66, ''],
 [67, ''],
 [68, ''],
 [69, ''],
 [70, ''],
 [71, ''],
 [72, ''],
 [73, ''],
 [74, ''],
 [75, ''],
 [76, ''],
 [77, ''],
 [78, ''],
 [79, ''],
 [80, ''],
 [81, ''],
 [82, ''],
 [83, ''],
 [84, ''],
 [85, ''],
 [86, ''],
 [87, ''],
 [88, ''],
 [89, ''],
 [90, ''],
 [91, ''],
 [92, ''

In [ ]:
import pandas as pd
import numpy as np
outdf = pd.DataFrame(np.array(df),columns=['id','answer'])
outdf.to_csv('./output.csv',index=False,header=True)